# In this notebook we will print the utilities_dataset.py

In the code below we use the open source database available here (https://github.com/f1db/f1db/tree/main)


To download the repo, run on the terminal the following commands : 

- wget https://github.com/f1db/f1db/archive/refs/heads/main.zip


- unzip main.zip -d f1db_data


In [1]:
from utilities_dataset import(
    load_circuit_specs, 
    load_and_process_circuit_specs,
    build_race_schedule,
    summarize_circuit,
    create_unique_circuits,
    add_drs_to_circuits,
    process_round_specs,
    discover_races_in_cache,
    build_race_output_dir,
    extract_drivers_for_race,
    process_lap_timing_data,
    extract_timing_app_data,
    align_and_clean_timing_data,
    merge_lap_timing_data, 
    merge_race_data_with_weather,
    get_session_keys, 
    fetch_race_control_data, 
    process_race_control_data,
    extract_rc_violations,
    merge_violations_with_timing,
    merge_race_flags_to_violations,
    combine_yearly_results,
    clean_combined_data,
    fetch_openf1_pit_and_stints_for_years,
    merge_pit_stops_to_clean_dataset,
    merge_stints_to_clean_dataset
)

import pandas as pd
import os
import json
from urllib.request import urlopen

In [2]:
circuits_folder = "/Users/jacoposinigaglia/Desktop/svizzera/terzo_semestre/Advanced_Data_Analysis/Progetto/f1db-main/src/data/circuits"
output_csv = "csv_output/circuit_specs.csv"

circuit_specs = load_circuit_specs(circuits_folder, output_path=output_csv)
circuit_specs.head()

,circuitRef,name,countryId,length_km,turns,drs_zones,width_m,type
0,east-london,Prince George,south-africa,3.920,9,None,None,RACE
1,nivelles,Nivelles-Baulers,belgium,3.724,7,None,None,RACE
2,monsanto,Monsanto,portugal,5.440,9,None,None,STREET
3,yas-marina,Yas Marina,united-arab-emirates,5.281,16,None,None,RACE
4,austin,Americas,united-states-of-america,5.513,20,None,None,RACE


In [3]:
circuit_specs = load_and_process_circuit_specs(output_csv)
circuit_specs.head()


,circuitRef,name,countryId,length_km,turns,type,num_drs_zones
0,east-london,Prince George,south-africa,3.920,9,RACE,0
1,nivelles,Nivelles-Baulers,belgium,3.724,7,RACE,0
2,monsanto,Monsanto,portugal,5.440,9,STREET,0
3,yas-marina,Yas Marina,united-arab-emirates,5.281,16,RACE,2
4,austin,Americas,united-states-of-america,5.513,20,RACE,2


In [4]:
schedule_years = range(2023, 2026)
merged_all_years= build_race_schedule(schedule_years, circuit_specs, cache_dir="./cache")

merged_all_years.head(10)


Configured to process years: [2023, 2024, 2025]
To process more years, check your cache folder and update schedule_years

Reading cache for 2023...
  ✓ Found 22 race folders
  ✓ Mapped 22/22 races to circuits

Reading cache for 2024...
  ✓ Found 24 race folders
  ✓ Mapped 24/24 races to circuits

Reading cache for 2025...
  ✓ Found 24 race folders
  ✓ Mapped 24/24 races to circuits

✓ Schedule file saved: csv_output/circuit_specs_with_rounds_2023_2025.csv
  Total race entries: 70
  Years: 2023-2025
  Unique circuits: 24


,circuitRef,year,round,name,countryId,length_km,turns,type,num_drs_zones
0,bahrain,2023,1,Bahrain,bahrain,5.412,15,RACE,3
1,jeddah,2023,2,Jeddah,saudi-arabia,6.174,27,STREET,3
2,melbourne,2023,3,Melbourne,australia,5.278,14,STREET,4
3,baku,2023,4,Baku,azerbaijan,6.003,20,STREET,2
4,miami,2023,5,Miami,united-states-of-america,5.412,19,STREET,3
5,monaco,2023,6,Monaco,monaco,3.337,19,STREET,1
6,catalunya,2023,7,Catalunya,spain,4.657,14,RACE,2
7,montreal,2023,8,Gilles Villeneuve,canada,4.361,13,STREET,3
8,spielberg,2023,9,Red Bull Ring,austria,4.326,10,RACE,3
9,silverstone,2023,10,Silverstone,united-kingdom,5.891,18,RACE,2


In [5]:

'''''

# Enable cache
fastf1.Cache.enable_cache("./cache")

# Download data for 2018-2024
YEARS = range(2025, 2026)
start = datetime.now()

for year in YEARS:
    print(f"\n{'='*60}\nYear: {year}\n{'='*60}")
    schedule = fastf1.get_event_schedule(year)
    races = schedule[schedule['RoundNumber'] >= 1]
    
    for idx, race in races.iterrows():
        try:
            print(f"[{race['RoundNumber']:2d}] {race['EventName']:40s}", end=" ")
            
            # Download Race + Qualifying
            fastf1.get_session(year, race['EventName'], 'R').load()
            fastf1.get_session(year, race['EventName'], 'Q').load()
            
            print("✓")
        except Exception as e:
            print(f"✗ {str(e)[:40]}")

print(f"\n Done! Time: {datetime.now() - start}")

'''


'\'\'\n\n# Enable cache\nfastf1.Cache.enable_cache("./cache")\n\n# Download data for 2018-2024\nYEARS = range(2025, 2026)\nstart = datetime.now()\n\nfor year in YEARS:\n    print(f"\n{\'=\'*60}\nYear: {year}\n{\'=\'*60}")\n    schedule = fastf1.get_event_schedule(year)\n    races = schedule[schedule[\'RoundNumber\'] >= 1]\n\n    for idx, race in races.iterrows():\n        try:\n            print(f"[{race[\'RoundNumber\']:2d}] {race[\'EventName\']:40s}", end=" ")\n\n            # Download Race + Qualifying\n            fastf1.get_session(year, race[\'EventName\'], \'R\').load()\n            fastf1.get_session(year, race[\'EventName\'], \'Q\').load()\n\n            print("✓")\n        except Exception as e:\n            print(f"✗ {str(e)[:40]}")\n\nprint(f"\n Done! Time: {datetime.now() - start}")\n\n'

In [6]:
# Geometry section
#=============================================================================
# DATA COLLECTION FROM CACHE
# =============================================================================
# This section processes ONLY pre-cached FastF1 data from ./cache folder
#
# TIP: If you're getting many failures, process year-by-year:
#    1. Look at your cache folder to see which years have data
#    2. Update SCHEDULE_YEARS above (line 200) and print statement above to process one year at a time
#    3. Run the script multiple times for different year ranges
#    4. Manually merge the output CSV files afterwards
#
# Example: Process only 2018:  SCHEDULE_YEARS = range(2018, 2019)
# Example: Process 2018-2019:  SCHEDULE_YEARS = range(2018, 2020)
# =============================================================================

print("\n" + "="*70)
print("PROCESSING CACHED DATA - No downloads will occur")
print("="*70)

all_data = []
total_races = len(merged_all_years)
success_count = 0
fail_count = 0

for idx, row in merged_all_years.iterrows():
    circuit_name = row["circuitRef"].capitalize()
    year = int(row["year"])
    round_num = int(row["round"])
    
    print(f"\n[{idx+1}/{total_races}] Processing: {year} Round {round_num:2d} - {row['name']}")

    try:
        df_tel = summarize_circuit(
            year,
            circuit_name,
            official_length_km=row["length_km"]  # Use static length if provided
        )

        # Add metadata
        df_tel["circuitRef"] = row["circuitRef"]
        df_tel["year"] = year
        df_tel["round"] = round_num
        df_tel["type"] = row["type"]
        df_tel["num_drs_zones"] = row["num_drs_zones"]
        df_tel["name"] = row["name"]

        all_data.append(df_tel)
        success_count += 1
        print(f"  ✓ Success")

    except Exception as e:
        fail_count += 1
        print(f"  ✗ Failed: {str(e)[:80]}")

# Summary
print("\n" + "="*70)
print(f"PROCESSING COMPLETE")
print(f"Successfully processed: {success_count}/{total_races} races")
print(f"Failed: {fail_count}/{total_races} races")
print("="*70)

if len(all_data) == 0:
    print("\n WARNING: No data was successfully processed!")
    print("Make sure the cache folder contains the required FastF1 data.")
else:
    telemetry_all_years = pd.concat(all_data, ignore_index=True)

    # Column ordering section::
    ordered_cols = [
        "circuitRef", "year", "round", "name", "type", "num_drs_zones", "length_km", "num_turns",
        "slow_corners", "medium_corners", "fast_corners", "flat_corners", "slow_cluster_max",
        "straight_distance_m", "straight_ratio", "straight_len_max_m", "n_major_straights",
        "heavy_braking_zones", "heavy_braking_mean_dv_kmh", "hb_spacing_std_m", "hb_at_end_of_max",
        "corner_density_tpkm", "avg_corner_angle", "total_corner_angle", "avg_corner_distance"
    ]
    telemetry_all_years = telemetry_all_years[[c for c in ordered_cols if c in telemetry_all_years.columns]]

    # Save the final dataset
    outdir = "csv_output"
    os.makedirs(outdir, exist_ok=True)
    
    # Create filename based on actual years in final dataset
    min_year = int(telemetry_all_years['year'].min())
    max_year = int(telemetry_all_years['year'].max())
    output_path = os.path.join(outdir, f"Circuit_geometry_{min_year}_{max_year}.csv")
    telemetry_all_years.to_csv(output_path, index=False)
    
    print(f"\n✓ Dataset saved to: {output_path}")
    print(f"  Total records: {len(telemetry_all_years)}")
    print(f"  Years covered: {min_year} - {max_year}")
    print(f"  Unique circuits: {telemetry_all_years['circuitRef'].nunique()}")


PROCESSING CACHED DATA - No downloads will occur

[1/70] Processing: 2023 Round  1 - Bahrain


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driv

  ✓ Success

[2/70] Processing: 2023 Round  2 - Jeddah


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
core        WARNING 	Driver 11 completed the race distance 00:00.035000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['11', '1', '14', '63', '44', '55', '16', '31', '10', '20', '22', '27', '24', '21', '81', '2', '4', '77', '23', '18']
core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


  ✓ Success

[3/70] Processing: 2023 Round  3 - Melbourne


...

  ✓ Success

[69/70] Processing: 2025 Round 23 - Lusail


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
core           INFO 	Finished loading data for 20 drivers: ['1', '81', '55', '4', '12', '63', '14', '16', '30', '22', '23', '44', '5', '43', '31', '10', '18', '6', '87', '27']
events      WARNING 	Correcting user input 'Yas-marina' to 'Singapore Grand Prix'
core           INFO 	Loading data for Singapore Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


  ✓ Success

[70/70] Processing: 2025 Round 24 - Yas Marina


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
core           INFO 	Finished loading data for 20 drivers: ['63', '1', '4', '81', '12', '16', '14', '44', '87', '55', '6', '22', '18', '23', '30', '43', '5', '31', '10', '27']


  ✓ Success

PROCESSING COMPLETE
Successfully processed: 70/70 races
Failed: 0/70 races

✓ Dataset saved to: csv_output/Circuit_geometry_2023_2025.csv
  Total records: 70
  Years covered: 2023 - 2025
  Unique circuits: 24


In [7]:
#--------------------------------------
# creating the unique circuits dataset
#--------------------------------------


# Before the unique circuits were more as the analysis was based from 2018 to 2025. Now it is only from 2023 to 2025.

input_file = "csv_output/Circuit_geometry_2023_2025.csv"
df_unique = create_unique_circuits(input_file)

df_unique


Dataset saved to: csv_output/Circuit_specs_unique.csv (24 circuits)


,circuitRef,year,round,name,type,num_drs_zones,length_km,num_turns,slow_corners,medium_corners,...,straight_len_max_m,n_major_straights,heavy_braking_zones,heavy_braking_mean_dv_kmh,hb_spacing_std_m,hb_at_end_of_max,corner_density_tpkm,avg_corner_angle,total_corner_angle,avg_corner_distance
0,melbourne,2025,1,Melbourne,STREET,4,5.278,14,5,4,...,627.7,0,5,152.1,1018.7,True,2.65,-0.4,-5.4,374.2
1,shanghai,2025,2,Shanghai,RACE,2,5.451,16,5,6,...,1117.3,1,6,204.8,648.9,True,2.94,-6.8,-108.7,336.8
2,suzuka,2025,3,Suzuka,RACE,1,5.807,18,6,6,...,1239.3,1,6,168.1,881.7,False,3.10,-176.6,-3178.4,320.4
3,bahrain,2025,4,Bahrain,RACE,3,5.412,15,5,5,...,984.3,1,10,165.2,353.2,False,2.77,-216.5,-3247.3,358.1
4,jeddah,2025,5,Jeddah,STREET,3,6.174,27,9,9,...,855.9,1,6,179.9,745.6,True,4.37,-58.7,-1585.9,225.0
5,miami,2025,6,Miami,STREET,3,5.412,19,6,7,...,1156.9,1,4,196.0,933.5,True,3.51,-73.8,-1401.3,280.3
6,imola,2025,7,Enzo e Dino Ferrari,RACE,1,4.909,19,6,7,...,340.5,0,9,148.5,498.3,True,3.87,-32.8,-622.7,257.1
7,monaco,2025,8,Monaco,STREET,1,3.337,19,6,7,...,131.2,0,10,149.8,330.9,False,5.69,13.8,262.2,172.9
8,catalunya,2025,9,Catalunya,RACE,2,4.657,14,5,4,...,627.7,0,5,152.1,1018.7,True,3.01,-0.4,-5.4,374.2
9,montreal,2025,10,Gilles Villeneuve,STREET,3,4.361,14,5,4,...,588.3,0,12,171.4,351.5,True,3.21,-16.0,-224.1,306.8


In [8]:
#-------------------------------------------------------------
# Adding the DRS specification the the unique circuits dataset
#-------------------------------------------------------------

add_drs_to_circuits(
    input_path="csv_output/Circuit_specs_unique.csv",
    output_path="csv_output/Circuit_geometry_unique.csv",
    cache_dir="./cache"
)

core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '87'


Reading csv_output/Circuit_specs_unique.csv...

Processing 24 circuits...

[1/24] 2025 melbourne... 

core        WARNING 	Fixed incorrect tyre stint information for driver '30'
core        WARNING 	Fixed incorrect tyre stint information for driver '5'
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 4 completed the race distance 00:00.022000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', '22', '31', '87', '30', '5', '14', '55', '7', '6']
core           INFO 	Loading data for Chinese Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached

✓ 3 zones, 2687m 
[2/24] 2025 shanghai... 

req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['81', '4', '63', '1', '31', '12', '23', '87', '18', '55', '6', '30', '7', '5', '27', '22', '14', '16', '44', '10']
core           INFO 	Loading data for Japanese Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


...

req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '81', '55', '4', '12', '63', '14', '16', '30', '22', '23', '44', '5', '43', '31', '10', '18', '6', '87', '27']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


✓ 3 zones, 1101m ✓longest
[24/24] 2025 yas-marina... 

req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '81', '4', '16', '63', '14', '31', '44', '27', '18', '5', '87', '55', '22', '12', '23', '6', '30', '10', '43']


✓ 2 zones, 1842m ✓longest

✓ Complete! Saved to csv_output/Circuit_geometry_unique.csv

Summary:
  Total: 24
  With DRS: 24
  On longest straight: 17


,circuitRef,year,round,name,type,num_drs_zones,length_km,num_turns,slow_corners,medium_corners,...,heavy_braking_mean_dv_kmh,hb_spacing_std_m,hb_at_end_of_max,corner_density_tpkm,avg_corner_angle,total_corner_angle,avg_corner_distance,drs_zones_detected,drs_total_len_m,drs_on_max
0,melbourne,2025,1,Melbourne,STREET,4,5.278,14,5,4,...,152.1,1018.7,True,2.65,-0.4,-5.4,374.2,3.0,2686.755960,False
1,shanghai,2025,2,Shanghai,RACE,2,5.451,16,5,6,...,204.8,648.9,True,2.94,-6.8,-108.7,336.8,3.0,1760.362488,True
2,suzuka,2025,3,Suzuka,RACE,1,5.807,18,6,6,...,168.1,881.7,False,3.10,-176.6,-3178.4,320.4,2.0,1175.563481,False
3,bahrain,2025,4,Bahrain,RACE,3,5.412,15,5,5,...,165.2,353.2,False,2.77,-216.5,-3247.3,358.1,4.0,2025.010707,True
4,jeddah,2025,5,Jeddah,STREET,3,6.174,27,9,9,...,179.9,745.6,True,4.37,-58.7,-1585.9,225.0,4.0,2101.783413,False
5,miami,2025,6,Miami,STREET,3,5.412,19,6,7,...,196.0,933.5,True,3.51,-73.8,-1401.3,280.3,4.0,2621.906739,True
6,imola,2025,7,Enzo e Dino Ferrari,RACE,1,4.909,19,6,7,...,148.5,498.3,True,3.87,-32.8,-622.7,257.1,2.0,1385.497371,True
7,monaco,2025,8,Monaco,STREET,1,3.337,19,6,7,...,149.8,330.9,False,5.69,13.8,262.2,172.9,3.0,1238.140770,False
8,catalunya,2025,9,Catalunya,RACE,2,4.657,14,5,4,...,152.1,1018.7,True,3.01,-0.4,-5.4,374.2,3.0,1275.899640,True
9,montreal,2025,10,Gilles Villeneuve,STREET,3,4.361,14,5,4,...,171.4,351.5,True,3.21,-16.0,-224.1,306.8,2.0,842.942250,False


Building the second dataset ( we analyze 2023, 2024 and 2025 separatelly, reusing the same code)

Every race has this datas:

- car_data.ff1pkl: <class 'dict'>
- weather_data.ff1pkl: <class 'dict'>
- driver_info.ff1pkl: <class 'dict'>
- position_data.ff1pkl: <class 'dict'>
- lap_count.ff1pkl: <class 'dict'>
- session_status_data.ff1pkl: <class 'dict'>
- race_control_messages.ff1pkl: <class 'dict'>
- timing_app_data.ff1pkl: <class 'dict'>
- track_status_data.ff1pkl: <class 'dict'>
- session_info.ff1pkl: <class 'dict'>
- extended_timing_data.ff1pkl: <class 'dict'>

Here we will cobine the extendend driving info , timing data ,timing data , and the weather datas


In [66]:
#-------------------------
#Loading the rounds dataset 
#-------------------------

round_specs = process_round_specs(
    input_csv='csv_output/circuit_specs_with_rounds_2023_2025.csv',
    output_csv='csv_output/Rounds_2023_2025.csv'
)

In [67]:
races_2025 = discover_races_in_cache(years=[2025])

Scanning cache for Race sessions in years: ['2025']
✓ Discovered 24 Race sessions



In [68]:
#----------------------------------------
# Build output directories for each race
#----------------------------------------

for race in races_2025:
    output_info = build_race_output_dir(race)


In [69]:
#--------------------------------------------------------------------
# Extract drivers for EVERY RACE in the season using helper functions
#--------------------------------------------------------------------

all_drivers_by_race = {}
for race in races_2025:
    drivers_info = extract_drivers_for_race(race['session_path'])
    all_drivers_by_race[race['race_name']] = drivers_info


Here is worth noticing that there is a commented part , the commented code is needed when we skip some races 

- in 2023 we skip the Dutch Grand Prix 

- in 2024 we skip the Las Vegas Grand Prix 

In [70]:
# The commented one do it for every race , in the uncommented one we skip the dutch grand prix as it has issues

# Process lap timing data for EVERY RACE and save to SINGLE file
all_lap_data_list = []
for race in races_2025:
    race_name = race['race_name']
    drivers_info = all_drivers_by_race[race_name]
    
    # Process lap timing data (no output_path to avoid saving individual files)
    lap_data = process_lap_timing_data(
        base_path=race['session_path'],
        drivers_info=drivers_info,
        output_path=None
    )
    all_lap_data_list.append(lap_data)
    print(f"✓ {race_name}")

# Combine all races into ONE dataframe
all_lap_data_combined = pd.concat(all_lap_data_list, ignore_index=True)

# Save to single file
all_lap_data_combined.to_excel('csv_output/2025_All_Races_Lap_Times.xlsx', index=False, sheet_name='Lap Data')
print(f"\n✓ Processed and saved {len(all_lap_data_list)} races to: csv_output/2025_All_Races_Lap_Times.xlsx")



✓ Merged 921 records with position and interval data
✓ 2025-03-16_Australian_Grand_Prix
✓ Merged 1065 records with position and interval data
✓ 2025-03-23_Chinese_Grand_Prix
✓ Merged 1059 records with position and interval data
✓ 2025-04-06_Japanese_Grand_Prix
✓ Merged 1128 records with position and interval data
✓ 2025-04-13_Bahrain_Grand_Prix
✓ Merged 897 records with position and interval data
✓ 2025-04-20_Saudi_Arabian_Grand_Prix
✓ Merged 1002 records with position and interval data
✓ 2025-05-04_Miami_Grand_Prix
✓ Merged 1205 records with position and interval data
✓ 2025-05-18_Emilia_Romagna_Grand_Prix
✓ Merged 1424 records with position and interval data
✓ 2025-05-25_Monaco_Grand_Prix
✓ Merged 1202 records with position and interval data
✓ 2025-06-01_Spanish_Grand_Prix
✓ Merged 1347 records with position and interval data
✓ 2025-06-15_Canadian_Grand_Prix
✓ Merged 1124 records with position and interval data
✓ 2025-06-29_Austrian_Grand_Prix
✓ Merged 822 records with position and i

In [ ]:

# # # Process lap timing data for every race 
# # all_lap_data_list = []
# # skipped_races = []

# for race in races_2025:
#     race_name = race['race_name']
    
#     # Skip races that don't work
#     if 'Las_Vegas_Grand_Prix' in race_name:
#         skipped_races.append(race_name)
#         print(f"⊘ SKIPPED {race_name}")
#         continue
    
#     drivers_info = all_drivers_by_race[race_name]
    
#     # Process lap timing data (no output_path to avoid saving individual files)
#     lap_data = process_lap_timing_data(
#         base_path=race['session_path'],
#         drivers_info=drivers_info,
#         output_path=None
#     )
#     all_lap_data_list.append(lap_data)
#     print(f"✓ {race_name}")

# # Combine all races into ONE dataframe
# all_lap_data_combined = pd.concat(all_lap_data_list, ignore_index=True)

# # Save to single file
# all_lap_data_combined.to_excel('csv_output/2024_All_Races_Lap_Times.xlsx', index=False, sheet_name='Lap Data')
# print(f"\n✓ Processed and saved {len(all_lap_data_list)} races to: csv_output/2024_All_Races_Lap_Times.xlsx")
# if skipped_races:
#     print(f"⊘ Skipped {len(skipped_races)} races: {', '.join([r.split('_')[0] for r in skipped_races])}")

In [71]:
#---------------------------------------
# Extract timing app data for every race
#---------------------------------------

all_timing_app_data = {}
for race in races_2025:
    race_name = race['race_name']
    
    # # Skip races that don't work
    # if 'Las_Vegas_Grand_Prix' in race_name:
    #     continue
    
    timing_app_path = os.path.join(race['session_path'], "timing_app_data.ff1pkl")
    
    # Extract timing data (not saving to file, storing in memory)
    timing_df = extract_timing_app_data(
        timing_app_path=timing_app_path,
        output_csv_path=None
    )
    all_timing_app_data[race_name] = timing_df

# Combine all races into ONE dataframe WITH RACE IDENTIFICATION
all_timing_app_list = []
for race_name, timing_df in all_timing_app_data.items():
    # Extract clean race name (remove date prefix: "2023-09-03_Italian_Grand_Prix" -> "Italian_Grand_Prix")
    clean_race_name = '_'.join(race_name.split('_')[1:])
    timing_df['race_name'] = clean_race_name
    all_timing_app_list.append(timing_df)

all_timing_app_combined = pd.concat(all_timing_app_list, ignore_index=True)

# Reorder columns to put race_name first
cols = all_timing_app_combined.columns.tolist()
cols.remove('race_name')
all_timing_app_combined = all_timing_app_combined[['race_name'] + cols]

# Save to single CSV file
all_timing_app_combined.to_csv('csv_output/2025_All_Races_Timing_App_Data.csv', index=False)
print(f"\n Extracted and saved timing app data for {len(all_timing_app_data)} races to: csv_output/2025_All_Races_Timing_App_Data.csv")


 Extracted and saved timing app data for 24 races to: csv_output/2025_All_Races_Timing_App_Data.csv


In [72]:
#----------------------------------------------
# Align and clean timing data file just created
#----------------------------------------------

all_timing_cleaned = {}
for race_name, timing_df in all_timing_app_data.items():
    timing_clean = align_and_clean_timing_data(
        timing_df=timing_df,
        output_csv_path=None,
        verbose=False,
        drop_columns=['TotalLaps']
    )
    all_timing_cleaned[race_name] = timing_clean

# Combine all races into ONE dataframe WITH RACE IDENTIFICATION
all_timing_cleaned_list = []
for race_name, timing_df in all_timing_cleaned.items():
    # Extract clean race name (remove date prefix: "2023-09-03_Italian_Grand_Prix" -> "Italian_Grand_Prix")
    clean_race_name = '_'.join(race_name.split('_')[1:])
    timing_df['race_name'] = clean_race_name
    all_timing_cleaned_list.append(timing_df)

all_timing_cleaned_combined = pd.concat(all_timing_cleaned_list, ignore_index=True)

# Reorder columns to put race_name first
cols = all_timing_cleaned_combined.columns.tolist()
cols.remove('race_name')
all_timing_cleaned_combined = all_timing_cleaned_combined[['race_name'] + cols]

# Save to single CSV file
all_timing_cleaned_combined.to_csv('csv_output/2025_All_Races_Timing_App_Data_cleaned.csv', index=False)
print(f"\n✓ Aligned and cleaned timing data for {len(all_timing_cleaned)} races to: csv_output/2025_All_Races_Timing_App_Data_cleaned.csv")


✓ Aligned and cleaned timing data for 24 races to: csv_output/2025_All_Races_Timing_App_Data_cleaned.csv


In [73]:
#------------------------------------------------------------------------
# Mergin the All_races time lap xlsx with the cleaned timing app data csv
#------------------------------------------------------------------------


# Build a dictionary of lap data from all_lap_data_list with race names as keys
all_lap_data_dict = {}
race_list = [race for race in races_2025] #if 'Las_Vegas_Grand_Prix' not in race['race_name']]
for idx, lap_data in enumerate(all_lap_data_list):
    if idx < len(race_list):
        race_full_name = race_list[idx]['race_name']
        all_lap_data_dict[race_full_name] = lap_data

# Merge lap times with cleaned timing data for ALL RACES (in memory to avoid race name mismatch)
all_merged_races = []

for race_full_name, lap_data in all_lap_data_dict.items():
    timing_data = all_timing_cleaned.get(race_full_name)
    
    if timing_data is None:
        continue
    
    # Merge by driver and lap number (same race, so no need for race identifier)
    merged = pd.merge(
        lap_data,
        timing_data,
        left_on=['RacingNumber', 'NumberOfLaps'],
        right_on=['Driver', 'LapNumber'],
        how='inner'
    )
    
    # Drop duplicate driver/lap columns
    if 'Driver' in merged.columns:
        merged = merged.drop(columns=['Driver'])
    if 'LapNumber' in merged.columns:
        merged = merged.drop(columns=['LapNumber'])
    
    all_merged_races.append(merged)

# Combine all merged races
merged_race_data = pd.concat(all_merged_races, ignore_index=True)

# Save to file
merged_race_data.to_excel('csv_output/2025_All_Races_Merged_Lap_Timing.xlsx', index=False, sheet_name='Merged Data')
print(f"\n✓ Merged lap timing data: {merged_race_data.shape[0]} rows, {merged_race_data.shape[1]} columns")
print(f"✓ Saved to: csv_output/2025_All_Races_Merged_Lap_Timing.xlsx")


✓ Merged lap timing data: 26076 rows, 19 columns
✓ Saved to: csv_output/2025_All_Races_Merged_Lap_Timing.xlsx


In [74]:
#-------------------------------------------------------------------------
# Merge weather data with lap timing data for races that have weather data
#-------------------------------------------------------------------------

all_merged_with_weather = []

for race_full_name, lap_data in all_lap_data_dict.items():
    # Get this race from races_2025 to get the session path
    matching_race = None
    for race in races_2025:
        if race['race_name'] == race_full_name:
            matching_race = race
            break
    
    if matching_race is None:
        continue
    
    # Get the actual circuit name from the lap data itself
    race_short_name = lap_data['name'].iloc[0] if 'name' in lap_data.columns and len(lap_data) > 0 else None
    
    if race_short_name is None:
        continue
    
    # Filter merged data for THIS RACE ONLY using the actual circuit name
    race_merged_data = merged_race_data[merged_race_data['name'] == race_short_name]
    
    if len(race_merged_data) == 0:
        continue
    
    # Merge with weather data
    try:
        # Check if weather file exists
        weather_file = os.path.join(matching_race['session_path'], 'weather_data.ff1pkl')
        if not os.path.exists(weather_file):
            continue
        
        race_with_weather = merge_race_data_with_weather(
            merged_race_data=race_merged_data,
            race_cache_path=matching_race['session_path'],
            tolerance_seconds=30
        )
        all_merged_with_weather.append(race_with_weather)
    except Exception as e:
        continue

# Check if we have any data to combine
if len(all_merged_with_weather) == 0:
    merged_race_data_with_weather = merged_race_data.copy()
else:
    # Combine all races with weather data
    merged_race_data_with_weather = pd.concat(all_merged_with_weather, ignore_index=True)

# Save to Excel file
output_file = 'csv_output/2025_All_Races_Merged_Lap_Timing_With_Weather.xlsx'
merged_race_data_with_weather.to_excel(output_file, index=False)
print(f"\n✓ Merged all races with weather data: {merged_race_data_with_weather.shape[0]} rows")
print(f"✓ Saved to: {output_file}")



✓ Merged all races with weather data: 26076 rows
✓ Saved to: csv_output/2025_All_Races_Merged_Lap_Timing_With_Weather.xlsx


#  OpenF1

From OpenF1 API  (https://openf1.org/), we will download the race control datas , then merge them with the Lap timing dataset


In [2]:

# Define years to process
years_to_process = [2023, 2024, 2025]

# Dictionary to store data for all years
all_rc_data = {}

for year in years_to_process:
    print(f"\n{'='*60}")
    print(f"Processing year: {year}")
    print(f"{'='*60}")
    
    try:
        # Get session keys
        session_keys = get_session_keys(year)
        print(f"✓ Found {len(session_keys)} sessions for {year}")
        
        # Fetch race control data
        df_rc = fetch_race_control_data(session_keys)
        df_rc.to_csv(f"csv_output/race_control_{year}_races.csv", index=False)
        print(f"✓ Saved raw data: csv_output/race_control_{year}_races.csv")
        
        # Store for later processing
        all_rc_data[year] = df_rc
        
    except Exception as e:
        print(f"✗ Error processing year {year}: {e}")
        continue


Processing year: 2023
✓ Found 22 sessions for 2023
[1/22] Fetching Sakhir (session_key=7953)... ✓ (73 events)
[2/22] Fetching Jeddah (session_key=7779)... ✓ (56 events)
[3/22] Fetching Melbourne (session_key=7787)... ✓ (103 events)
[4/22] Fetching Baku (session_key=9070)... ✓ (52 events)
[5/22] Fetching Miami (session_key=9078)... ✓ (38 events)
[6/22] Fetching Monte Carlo (session_key=9094)... ✓ (242 events)
[7/22] Fetching Catalunya (session_key=9102)... ✓ (47 events)
[8/22] Fetching Montreal (session_key=9110)... ✓ (75 events)
[9/22] Fetching Spielberg (session_key=9118)... ✓ (210 events)
[10/22] Fetching Silverstone (session_key=9126)... ✓ (44 events)
[11/22] Fetching Hungaroring (session_key=9133)... ✓ (85 events)
[12/22] Fetching Spa-Francorchamps (session_key=9141)... ✓ (54 events)
[13/22] Fetching Zandvoort (session_key=9149)... ✓ (74 events)
[14/22] Fetching Monza (session_key=9157)... ✓ (43 events)
[15/22] Fetching Singapore (session_key=9165)... ✓ (74 events)
[16/22] Fetchin

In [3]:

#  EXTRACT TRACK LIMIT VIOLATIONS AND OFF-TRACK INCIDENTS

print(f"\n{'='*60}")
print("Processing and saving results for all years...")
print(f"{'='*60}")

for year, df_rc in all_rc_data.items():
    try:
        # Extract violations FIRST on raw data (before processing)
        violations = extract_rc_violations(df_rc)
        violations.to_csv(f"csv_output/race_control_{year}_violations.csv", index=False)
        print(f"✓ Saved violations: csv_output/race_control_{year}_violations.csv ({len(violations)} records)")
        
        # Then process main race control flags
        rc_processed = process_race_control_data(df_rc)
        rc_processed.to_csv(f"csv_output/race_control_{year}_processed.csv", index=False)
        print(f"✓ Saved processed data: csv_output/race_control_{year}_processed.csv ({len(rc_processed)} records)")
        
    except Exception as e:
        print(f"✗ Error processing {year}: {e}")
        continue



Processing and saving results for all years...
✓ Saved violations: csv_output/race_control_2023_violations.csv (495 records)
✓ Saved processed data: csv_output/race_control_2023_processed.csv (511 records)
✓ Saved violations: csv_output/race_control_2024_violations.csv (350 records)
✓ Saved processed data: csv_output/race_control_2024_processed.csv (849 records)
✓ Saved violations: csv_output/race_control_2025_violations.csv (328 records)
✓ Saved processed data: csv_output/race_control_2025_processed.csv (725 records)


Mergin the All_Races_Merged_Lap_Timing_With_Weather with the race control violations first and then merge the obtained file with the race with control processed

In [4]:
merge_violations_with_timing()


Merging violations with lap timing data...

Processing year 2023...
  ✓ Saved: csv_output/2023_Merged_Lap_Timing_With_Violations.xlsx

Processing year 2024...
  ✓ Saved: csv_output/2024_Merged_Lap_Timing_With_Violations.xlsx

Processing year 2025...
  ✓ Saved: csv_output/2025_Merged_Lap_Timing_With_Violations.xlsx



In [5]:
# Merge the race control processed with violations dataset just obtained

result = merge_race_flags_to_violations(years=[2023, 2024, 2025])

In [6]:
# Step 3: Combine yearly results

combined = combine_yearly_results(years=[2023, 2024, 2025])


Combined 3 years into csv_output/Combined_Lap_Timing_Violations_RaceControl.xlsx
Total rows: 73,592


In [7]:
# Cleaning the combine dataset

if combined is not None:
    cleaned = clean_combined_data(combined)


Cleaned data saved to csv_output/Combined_Lap_Timing_Clean.xlsx
Total rows: 73,592
Final columns (24): ['year', 'round', 'name', 'Grand_Prix', 'Name', 'RacingNumber', 'Team', 'lap_number', 'LapTime', 'IntervalToPositionAhead', 'Position', 'Stint', 'Compound', 'New', 'LapInStint', 'AirTemp', 'Humidity', 'Pressure', 'TrackTemp', 'WindDirection', 'WindSpeed', 'Rainfall', 'any_violation', 'lap_clean']


As the OpenF1 API do not have datas for the pitstop for the first six races we have to infer them based on stint and compound.

In [8]:
df_pits, df_stints = fetch_openf1_pit_and_stints_for_years(
    years=(2023, 2024, 2025),
    save_csv=False  # Don't auto-save
)

# Drop columns you don't want
df_pits = df_pits.drop(columns=[
    "round", "race_session_key", "meeting_key", "date", "pit_duration", "session_key"
], errors='ignore')


df_stints = df_stints.drop(columns=["grand_prix", "race_session_key", "meeting_key", "session_key"], errors='ignore')

 # Sort by year (2023 first), then round (1,2,3...), then grand_prix_mapped, then driver_number
df_stints = df_stints.sort_values(by=["year", "round", "grand_prix_mapped", "driver_number"],ascending=[True, True, True, True]).reset_index(drop=True)

# Save manually
df_pits.to_csv("csv_output/openf1_pit_2023_2025.csv", index=False)
df_stints.to_csv("csv_output/openf1_stints_2023_2025.csv", index=False)


Year 2023: fetching race sessions… 22 sessions
  [1/22] Sakhir (session_key=7953)
      Pits … ✓ 0 | Stints … ✓ 70
  [2/22] Jeddah (session_key=7779)
      Pits … ✓ 0 | Stints … ✓ 44
  [3/22] Melbourne (session_key=7787)
      Pits … ✓ 0 | Stints … ✓ 85
  [4/22] Baku (session_key=9070)
      Pits … ✓ 0 | Stints … ✓ 43
  [5/22] Miami (session_key=9078)
      Pits … ✓ 0 | Stints … ✓ 40
  [6/22] Monte Carlo (session_key=9094)
      Pits … ✓ 0 | Stints … ✓ 57
  [7/22] Catalunya (session_key=9102)
      Pits … ✓ 43 | Stints … ✓ 63
  [8/22] Montreal (session_key=9110)
      Pits … ✓ 33 | Stints … ✓ 53
  [9/22] Spielberg (session_key=9118)
      Pits … ✓ 63 | Stints … ✓ 83
  [10/22] Silverstone (session_key=9126)
      Pits … ✓ 24 | Stints … ✓ 44
  [11/22] Hungaroring (session_key=9133)
      Pits … ✓ 36 | Stints … ✓ 56
  [12/22] Spa-Francorchamps (session_key=9141)
      Pits … ✓ 37 | Stints … ✓ 57
  [13/22] Zandvoort (session_key=9149)
      Pits … ✓ 101 | Stints … ✓ 121
  [14/22] Monza (s

In [3]:
# Load your cleaned dataset
print("Loading the combined lap timing cleaned dataset")
df_clean = pd.read_excel("csv_output/Combined_Lap_Timing_Clean.xlsx")

# Drop old columns that will be replaced by OpenF1 data
df_clean = df_clean.drop(columns=["Stint", "Compound", "New", "LapInStint"], errors='ignore')

# Merge stint data (adds: Stint, Compound, TyreAgeAtStart, LapInStint)
print(f"\nMerging stint data")
df_with_stints = merge_stints_to_clean_dataset(df_clean)

# Merge pit stops (adds: PitStop, Pit_Out)
print(f"\nMerging pit stops")
df_with_pits = merge_pit_stops_to_clean_dataset(df_with_stints)

# Save result
output_file = "csv_output/Combined_Lap_Timing_Final.xlsx"
df_with_pits.to_excel(output_file, index=False)
print(f"\n✓ Saved to: {output_file}")

Loading the combined lap timing cleaned dataset

Merging stint data
Loading stint data from csv_output/openf1_stints_2023_2025.csv...

Merging stint data...

✓ Matched laps: 73,583 (100.0%)
✗ Unmatched laps: 9
  Total laps: 73,592

Compound distribution:
  HARD: 34,665 (47.1%)
  MEDIUM: 27,179 (36.9%)
  SOFT: 7,710 (10.5%)
  INTERMEDIATE: 3,873 (5.3%)
  WET: 81 (0.1%)
  nan: 75 (0.1%)
  <NA>: 9 (0.0%)

Stint distribution:
  Stint 1: 23,232 (31.6%)
  Stint 2: 29,810 (40.5%)
  Stint 3: 15,841 (21.5%)
  Stint 4: 3,472 (4.7%)
  Stint 5: 884 (1.2%)
  Stint 6: 312 (0.4%)
  Stint 7: 32 (0.0%)
  Stint <NA>: 9 (0.0%)

Merging pit stops
Loading pit data from csv_output/openf1_pit_2023_2025.csv...

 6 races without OpenF1 data - using stint-based fallback:
   2023 Round 1 - Bahrain_Grand_Prix: 50 pit stops, 50 pit-outs (stint-based)
   2023 Round 2 - Saudi_Arabian_Grand_Prix: 24 pit stops, 24 pit-outs (stint-based)
   2023 Round 3 - Australian_Grand_Prix: 64 pit stops, 8 pit-outs (stint-based)
  